In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
from matplotlib.ticker import FormatStrFormatter
import matplotlib.gridspec as gridspec


from nest import detectors
from nest import pls
from nest import overlap 
from nest import nell 
from nest import skymap

from nest.skymap import Skymaps
from nest.overlap import Response
from nest.nell import AngularResponse

%config InlineBackend.figure_format = 'retina'

from astropy.cosmology import Planck15
cosmo = Planck15
H0 =  cosmo.H0.to('1/s').value

c = 299792458 #m/s

plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size'] = 20
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['font.family'] = 'serif'

In [4]:
# SNR 

def power_law_noise(f, f_pivot_noise, N_pivot, r, n_noise):
    '''
    Power-law correlated noise (i.e. Newtonian noise from seismic body waves)
    '''
    return N_pivot * r * (f/f_pivot_noise)**n_noise


def N_aet(f, N_auto_f, f_pivot_noise, N_pivot, r, n_noise):
    '''
    Noise in the diagonal basis, general for triangular-shaped detectors 
    '''
    N_cross_f = power_law_noise(f, f_pivot_noise, N_pivot, r, n_noise)
    N_aet = np.zeros((3, len(f)))
    N_aet[0] = N_auto_f -   N_cross_f
    N_aet[1] = N_auto_f -   N_cross_f
    N_aet[2] = N_auto_f + 2*N_cross_f
    return N_aet


def SNR(Tobs, f):
    f_full, PSD_full = detectors.detector_Pn('ET A')
    PSD = np.interp(f,f_full, PSD_full)
    gamma = Response.overlap('ET A', 'ET A', f, 0, 't')  - Response.overlap('ET A', 'ET B', f, 0, 't') 
    signal = 1e-12*(f/25)**(2/3)
    integrand = signal**2 * gamma**2 / (PSD**2 * f**6)
    return np.sqrt(4 * Tobs * np.trapz(integrand, f) * (3*H0**2/10*np.pi**2)**2)

In [5]:
Tobs = 24*60*60*365
f = np.logspace(0, np.log10(200), 1000)

snr = SNR(Tobs, f)


In [6]:
print(snr)
print(1/snr)

print(1/np.sqrt(snr))

(175.29796073919056+7.063790736648401e-19j)
(0.005704572921346224-2.2987095336661262e-23j)
(0.07552862319244423-1.5217472770615037e-22j)
